In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from pydantic import create_model
import inspect,json
from inspect import Parameter
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.utils.function_calling import (
    convert_to_openai_function,
    convert_to_openai_tool,
)
from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

import random
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain_community.tools import BaseTool, StructuredTool, tool
print(load_dotenv())
import os
import openai

client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

True


## Building Custom Agent

Defining the function

In [2]:
class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")

class CustomCalculatorTool(BaseTool):
    name = "Calculator"
    description = "Compute a^2 - b^2 that is 'difference of square of numbers' between two numbers"
    args_schema: Type[BaseModel] = CalculatorInput
    return_direct: bool = True

    def _run(
        self, a: int, b: int, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return a**2 - b**2

In [3]:
search = CustomCalculatorTool()
print(search.name)
print(search.description)
print(search.args)

Calculator
Compute a^2 - b^2 that is 'difference of square of numbers' between two numbers
{'a': {'title': 'A', 'description': 'first number', 'type': 'integer'}, 'b': {'title': 'B', 'description': 'second number', 'type': 'integer'}}


In [8]:
from langchain_core.tools import tool
# @tool(return_direct=True)
def squared_num_diff(num1:int, num2:int) -> int:
    "Compute a^2 - b^2 that is 'difference of square of numbers' between two numbers"
    return num1**2 - num2**2
    

In [5]:
search.run({'a':3,'b':2})

5

In [9]:
squared_num_diff(3,2)

5

In [17]:
tool_squared_num_diff = convert_to_openai_tool(squared_num_diff)

In [18]:
tool_squared_num_diff

{'type': 'function',
 'function': {'name': 'squared_num_diff',
  'description': "Compute a^2 - b^2 that is 'difference of square of numbers' between two numbers",
  'parameters': {'type': 'object',
   'properties': {'num1': {'type': 'integer'}, 'num2': {'type': 'integer'}},
   'required': ['num1', 'num2']}}}

In [23]:
MODEL = 'llama3-groq-70b-8192-tool-use-preview'
user_prompt = "what is 4^square -1^square?"
user_prompt = "who is president of india ?"

messages=[
        {
            "role": "system",
            "content": "You are a math assistant. Use the available tools to answer question if it requires use of tool else answer normally."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
response = client.chat.completions.create(
        model=MODEL, # LLM to use
        messages=messages, # Conversation history
        # stream=False
        tools=[tool_squared_num_diff], # Available tools (i.e. functions) for our LLM to use
        tool_choice="auto", # Let our LLM decide when to use tools
        max_tokens=4096 # Maximum number of tokens to allow in our response
    )
print(response)

ChatCompletion(id='chatcmpl-1c80cab3-8a2e-41b9-b9ac-a28dc5bc2cef', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm sorry but I do not have the capability to perform this task for you, I am happy to help you with any other queries you may have.", role='assistant', function_call=None, tool_calls=None))], created=1731159717, model='llama3-groq-70b-8192-tool-use-preview', object='chat.completion', system_fingerprint='fp_ee4b521143', usage=CompletionUsage(completion_tokens=32, prompt_tokens=248, total_tokens=280, completion_time=0.101896846, prompt_time=0.018270727, queue_time=0.014971858999999997, total_time=0.120167573), x_groq={'id': 'req_01jc8jhx86e2rv593n83q7zjgh'})


In [25]:
response.choices.message

AttributeError: 'list' object has no attribute 'message'

In [20]:
from groq import Groq
import json

# Initialize the Groq client
client = Groq()
response = client.chat.completions.create(
        model=MODEL, # LLM to use
        messages=messages, # Conversation history
        # stream=False
        tools=[tool_squared_num_diff], # Available tools (i.e. functions) for our LLM to use
        tool_choice="auto", # Let our LLM decide when to use tools
        max_tokens=4096 # Maximum number of tokens to allow in our response
    )
print(response)

ChatCompletion(id='chatcmpl-e9d0dcb9-c2ae-4a6c-85af-e249abf9c001', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_7vcq', function=Function(arguments='{"num1": 4, "num2": 1}', name='squared_num_diff'), type='function')]))], created=1731159634, model='llama3-groq-70b-8192-tool-use-preview', object='chat.completion', system_fingerprint='fp_ee4b521143', usage=CompletionUsage(completion_tokens=36, prompt_tokens=253, total_tokens=289, completion_time=0.115536357, prompt_time=0.032082987, queue_time=0.0010418199999999989, total_time=0.147619344), x_groq={'id': 'req_01jc8jfcfae2ctjbtvyc589x6x'})
